In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import json
import requests
from tqdm import tqdm
import time
from concurrent.futures import ThreadPoolExecutor, as_completed
import numpy as np
import os


headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} 

In [27]:
links = ['https://www.theborneopost.com/category/news/sarawak/',
        'https://www.theborneopost.com/category/news/sabah/',
        'https://www.theborneopost.com/category/news/nation/',
        'https://www.theborneopost.com/category/world/',
        'https://www.theborneopost.com/category/business/',
        'https://www.theborneopost.com/category/sports/',
        'https://www.theborneopost.com/category/lite-stories/',
        'https://www.theborneopost.com/category/features/',
        'https://www.theborneopost.com/category/columns/',
        'https://www.theborneopost.com/category/uncategorized/']

In [29]:
def crawl(url):
    while True:
        try:
            r = requests.get(url,headers = headers)
            if r.status_code == 508:
                print(r.status_code)
                print('error in link, retrying... ', url)
                time.sleep(20)
                continue
            break
        except Exception as e:
            print(e)
            time.sleep(1.0)

    soup = BeautifulSoup(r.text, "lxml")
    
    block = soup.find_all("div",attrs = {"class":"content"})
    
    if block is None:
        return

    for link in soup.find_all("div",attrs = {"class":"content"}):

        href = link.find('a').get('href')

        hrefs.append(href)
    

In [ ]:
max_worker = 5
hrefs = []

for url in links:
    
    print(url)
    r = requests.get(url,headers = headers)
    soup = BeautifulSoup(r.content, "html.parser")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    pgno = [a_ for a_ in a if 'page' in a_]
    max_page = max([int(a_.split('page')[1].replace('/','')) for a_ in pgno])
    print(max_page)
    for i in tqdm(range(1, max_page + 1, max_worker)):
        aranged = np.arange(i, i + max_worker)
        urls = [f'{url}/page/{a}' for a in aranged]
        with ThreadPoolExecutor(max_workers=max_worker) as executor:
            futures = {executor.submit(crawl, url): url for url in urls}
        
        for future in as_completed(futures):
            future.result()


In [19]:
with open('theborneopost-link.json', 'a') as f:
    json.dump(hrefs, f)

In [39]:
with open('theborneopost-link.json') as f:
    hrefs = json.load(f)

In [44]:
def process_url(x):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36'} # This is chrome, you can set whatever browser you like

    
    while True:
        try:
            r = requests.get(x, headers=headers)
            break
        except Exception as e:
            print(e)
            time.sleep(5.0)
    
    soup = BeautifulSoup(r.text, "lxml")

    try:
        headline = soup.find('h1', class_="post-title item fn").text
        h = soup.find('div', attrs = {"class":"post-content description"})
        p = [p.text for p in h.find_all("p") if len(p.text.split(" ")) > 3]
    except Exception as e:
        print('error in link:'+ x)
        print(e)
        return None

    
    data = {'url': x, 'headline': headline, 'content': p}
    return data

In [ ]:
max_workers = 20

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(process_url, x) for x in hrefs]
    
    for future in tqdm(futures, total=len(hrefs)):
        result = future.result()
        if result:
            with open('theborneopost.jsonl', 'a',encoding='utf8') as f:
                json.dump(result, f,ensure_ascii=False)
                f.write('\n')

In [22]:
process_url(hrefs[0])

{'url': 'https://www.theborneopost.com/2021/03/14/dough-ing-it-her-way/',
 'headline': '\nDough-ing it her way ',
 'content': ['A TERUBOK fish, a unicorn, and a traditional lantern were seemingly unrelated items but they all had one thing in common.',
  'They were cakes lovingly crafted by cake artist Abigail Lim, who believes no design is unachievable unless it’s limited by the client’s budget or technically impossible.',
  'In such circumstances, Lim would explain to clients carefully and provide alternatives.',
  '“If the issue is budget, I can usually suggest alternatives that have the same feel clients are going for but simplified to be within their budget.',
  '“For the latter, I would explain it cannot be done that way. They usually understand when I suggest alternatives and assure them the taste wouldn’t be compromised,” she told thesundaypost.',
  'The former engineer has come a long way since setting up her baking business in 2012.',
  'Growing up, Lim was influenced by her m